In [1]:
import json
import requests
import re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from pandas import json_normalize
from datetime import datetime
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from cfuzzyset import cFuzzySet as FuzzySet

In [4]:
df = pd.read_csv('blackstone_operations.csv')
df

,Unnamed: 0,Three Months Ended 2023,Three Months Ended 2022
0,Investment income:,NaN,NaN
1,Interest income,254221000.0,1.709890e+08
2,Payment in-kind interest income,9841000.0,8.686000e+06
3,Dividend income,—,5.908000e+06
4,Fee income,876000.0,1.400000e+04
5,Total investment income,264938000.0,1.855970e+08
6,Expenses:,NaN,NaN
7,Interest expense,66728000.0,4.030100e+07
8,Management fees (Note 3),24696000.0,2.563600e+07
9,Income based incentive fees (Note 3),30393000.0,2.128400e+07


In [5]:
list_of_headers = [
    {
        'row_title': 'ASSETS',
        'search_strings': [], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Investments at fair value',
        'search_strings': [], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Non-controlled/non-affiliated investments',
        'search_strings': ['non controlled unaffiliated investments', 'non controlled non affiliated investments', 'non controlled non affiliate company investments', 'non controlled non affiliated company investments', 'non control non affiliate investments', 'non controlled non affiliate investments'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Controlled/affiliated investments',
        'search_strings': ['control investments ', 'controlled affiliate company investments', 'controlled affiliated investments'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Non-controlled/affiliated investments',
        'search_strings': ['non controlled affiliated investments', 'non controlled affiliate company investments', 'non controlled affiliated company investments', 'affiliate investments'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Total investments at fair value',
        'search_strings': ['total investments at fair value', 'total investments'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Cash',
        'search_strings': ['cash', 'cash and cash equivalents'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Restricted cash',
        'search_strings': ['restricted cash', 'restricted cash and cash equivalents'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Interest receivable',
        'search_strings': ['interest receivable', 'interest and fees receivable', 'interest and dividends receivable', 'interest and dividend receivable and other assets', 'interest receivable from non controlled non affiliated investments'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Receivable for investments sold',
        'search_strings': ['receivable for open trades', 'receivable from unsettled transactions', 'receivable for investments sold', 'receivable for investments sold and repaid', 'receivable for securities sold', 'receivables for:', 'receivable from a controlled affiliate'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Prepaid expenses and other assets',
        'search_strings': ['other assets', 'prepaid expenses and other assets', 'prepaid expenses'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Deferred financing costs',
        'search_strings': ['deferred financing fees', 'deferred financing costs', 'deferred financing costs on revolving credit facility'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Total assets',
        'search_strings': ['total assets'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'LIABILITIES',
        'search_strings': [], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Debt',
        'search_strings': ['debt'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Management fees payable',
        'search_strings': ['base management fees payable', 'management fees payable', 'management fee payable', 'management and incentive fees payable'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Payable for investments purchased',
        'search_strings': ['payable for investments purchased'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Dividends payable',
        'search_strings': ['dividends payble', 'dividend payable'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Incentive fees payable',
        'search_strings': ['capital gains incentive fees payable', 'incentive management fees payable', 'incentive fees payable'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Income incentive fees payable',
        'search_strings': ['subordinated income incentive fees payable', 'income based fees payable', 'income based incentive fees payable'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Interest payable',
        'search_strings': ['interest and facility fees payable', 'interest payable', 'interest and other debt expenses payable'], 
         'value1': None,
        'value2': None
    },
    {
        'row_title': 'Accounts payable and other liabilities',
        'search_strings': ['accounts payable and other liabilities', 'accounts payable and accrued liabilities'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Credit facilities',
        'search_strings': ['secured borrowings', 'borrowings under credit facilities', 'credit facilities', 'revolving credit facility'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Accrued expenses',
        'search_strings': ['accrued expenses', 'accrued offering costs', 'accrued expenses and other liabilities', 'accrued trustee fees', 'other accrued expenses and liabilities'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Total liabilities',
        'search_strings': ['total liabilities'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'NET ASSETS',
        'search_strings': [], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Common stock',
        'search_strings': ['common stock'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Preferred stock',
        'search_strings': ['preferred stock'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Paid-in capital in excess of par',
        'search_strings': ['paid in capital in excess of par', 'capital in excess of par value', 'additional paid in capital'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Total distributable earnings',
        'search_strings': ['accumulated undistributed overdistributed earnings', 'total distributable earnings loss', 'total undistributed earnings', 'total distributable loss earnings', 'retained earnings'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Total net assets',
        'search_strings': ['total net assets', 'total stockholders equity'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Total liabilites and net assets',
        'search_strings': ['total liabilities and stockholders  equity', 'total liabilities and net assets', 'total liabilities and total net assets'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Net assets per share',
        'search_strings': ['net assets per share', 'net asset value per share', 'net asset value per common share', 'net asset value per share of common stock at period end'], 
        'value1': None,
        'value2': None
    },
]

# Create a DataFrame from the list_of_headers
desired_columns = ['row_title', 'value1', 'value2']
df = pd.DataFrame([{key: d[key] for key in desired_columns} for d in list_of_headers])

print(df)

                                    row_title value1 value2
0                                      ASSETS   None   None
1                   Investments at fair value   None   None
2   Non-controlled/non-affiliated investments   None   None
3           Controlled/affiliated investments   None   None
4       Non-controlled/affiliated investments   None   None
5             Total investments at fair value   None   None
6                                        Cash   None   None
7                             Restricted cash   None   None
8                         Interest receivable   None   None
9             Receivable for investments sold   None   None
10          Prepaid expenses and other assets   None   None
11                   Deferred financing costs   None   None
12                               Total assets   None   None
13                                LIABILITIES   None   None
14                                       Debt   None   None
15                    Management fees pa

In [25]:
def clean_text(row):
    # Strip out special characters/punction
    text = row.lower().strip()
    stripped_string = re.sub(r'[^a-zA-Z0-9\s]+', ' ', text)
    return stripped_string

def average_score(row, terms):
    # find average fuzzy score among all variations of one row title
    total_score = 0
    for search_string in terms:
        score = fuzz.partial_ratio(row, search_string)
        total_score += score
    if len(terms) != 0:
        average_score = total_score / len(terms) 
    else:
        average_score = 0
    return average_score 
    
def return_best_match(row, type_list):
    # return row title of best match
    row = clean_text(row)
    highest_average = 0
    winner = pd.NA
    for header in type_list:
        score = average_score(row, header['search_strings'])
        if score > highest_average:
            highest_average = score
            winner = header['row_title']
    for header in type_list:
        for search_string in header['search_strings']:
            if search_string == row or search_string in row:
                winner = header['row_title']
                
    return winner

In [14]:
# open csv file and iterate over each row
df = pd.read_csv('main_street_capital_assets_and_liabilities.csv')

for index, row in df.iterrows():
    header = return_best_match(row.iloc[0], list_of_headers)
    first_value = row.iloc[1]
    sec_value = row.iloc[2]
    # Store the value in list_of_headers
    for item in list_of_headers:
        if item['row_title'] == header:
            item['value1'] = first_value
            item['value2'] = sec_value

# Create a DataFrame from the list_of_headers
desired_columns = ['row_title', 'value1', 'value2']
df1 = pd.DataFrame([{key: d[key] for key in desired_columns} for d in list_of_headers])

print(df1)

                                    row_title         value1        value2
0                                      ASSETS           None           NaN
1                   Investments at fair value           None           NaN
2   Non-controlled/non-affiliated investments  11861000000.0  1.202600e+10
3           Controlled/affiliated investments   1746003000.0  1.703172e+09
4       Non-controlled/affiliated investments   1788687000.0  1.780646e+09
5             Total investments at fair value   4160917000.0  4.102177e+09
6                                        Cash     39752000.0  4.912100e+07
7                             Restricted cash     18000000.0  3.400000e+07
8                         Interest receivable    275000000.0  2.270000e+08
9             Receivable for investments sold       505000.0  3.810000e+05
10          Prepaid expenses and other assets     84597000.0  8.273100e+07
11                   Deferred financing costs      7484000.0  7.475000e+06
12                       

In [1236]:
# Create table scraps
table_scraps = []
df2 = pd.DataFrame(table_scraps, columns=['row_title', 'value1', 'value2'])

for index, row in df.iterrows():
    count = 0
    for index1, row1 in df1.iterrows():
        if row.iloc[1] != row1.iloc[1] and row.iloc[2] != row1.iloc[2]:
            count += 1
            #print(str(row.iloc[1]) + str(row1.iloc[1]))
    if (count == len(df1)):
        if clean_text(row.iloc[0]) != 'assets' and clean_text(row.iloc[0]) != 'investments at fair value' and clean_text(row.iloc[0]) != 'investments  at fair value' and clean_text(row.iloc[0]) != 'liabilities' and clean_text(row.iloc[0]) != 'stockholders  equity' and clean_text(row.iloc[0]) != 'net assets':
            new_row = [row.iloc[0], row.iloc[1], row.iloc[2]]
            df2.loc[len(df2)] = new_row

print(df2)


                                           row_title        value1  \
0  Debt (net of debt issuance costs of $7,928 and...  1935409000.0   
1             Commitments and contingencies (Note 8)           NaN   

         value2  
0  2012660000.0  
1           NaN  


In [1237]:
operations_list = [
    {
        'row_title': 'EXPENSES',
        'search_strings': [], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Interest expense',
        'search_strings': ['interest and credit facility fees', 'interest and other financing fees', 'interest and other debt expenses', 'interest', 'interest and other debt financing expenses', 'interest expense'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Base management fees',
        'search_strings': ['base management fees', 'base management fee (Note 2)', 'management fees'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Incentive fees',
        'search_strings': ['capital gains incentive fees', 'incentive management fees', 'incentive fees', 'incentive fee', 'income incentive fee', 'subordinated income incentive fee', 'income based incentive fee'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Administrative fees',
        'search_strings': ['administrative fees', 'administrative service fee', 'administrative services expenses'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'General and administrative expenses',
        'search_strings': ['other general and administrative', 'general and administrative expenses', 'general and administrative', 'other general and administrative expenses'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Total expenses',
        'search_strings': ['total expenses', 'total operating expenses'], 
        'value1': None,
        'value2': None
    },
]

# Create a DataFrame from the operations_list
desired_columns = ['row_title', 'value1', 'value2']
op_df = pd.DataFrame([{key: d[key] for key in desired_columns} for d in operations_list])

print(op_df)

                             row_title value1 value2
0                             EXPENSES   None   None
1                     Interest expense   None   None
2                 Base management fees   None   None
3                       Incentive fees   None   None
4                  Administrative fees   None   None
5  General and administrative expenses   None   None
6                       Total expenses   None   None


In [214]:
investment_income_list = [
    {
        'row_title': 'Investment income',
        'search_strings': [], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Non-controlled, non-affiliated investments: Interest income',
        'search_strings': ['from non controlled non affiliate company investments interest income', 'from non controlled non affiliated investments interest income', 'non control non affiliate investments interest income', 'investment income from non controlled  non affiliated investments interest income'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Non-controlled, non-affiliated investments: Dividend income',
        'search_strings': ['from non controlled non affiliate company investments dividend income', 'from non controlled non affiliated investments dividend income', 'non control non affiliate investments dividend income', 'investment income from non controlled  non affiliated investments dividend income'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Non-controlled, non-affiliated investments: Payment-in-kind income',
        'search_strings': ['from non controlled non affiliate company investments payment in kind interest income', 'from non controlled non affiliated investments payment in kind interest income', 'non control non affiliate investments payment in kind interest income', 'investment income from non controlled  non affiliated investments payment in kind interest income'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Non-controlled, non-affiliated investments: Other income',
        'search_strings': ['from non controlled non affiliate company investments other income', 'from non controlled non affiliated investments other income', 'non control non affiliate investments other income', 'investment income from non controlled  non affiliated investments other income', 'from non controlled non affiliated investments fee income'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Non-controlled, non-affiliated investments: total interest, fee, and dividend income',
        'search_strings': ['interest fee and dividend income non control non affiliate investments'],
        'value1' : None,
        'value2' : None
    },
    {
        'row_title': 'Controlled affiliated investments: Interest income',
        'search_strings': ['from controlled affiliate company investments interest income', 'from controlled affiliated investments  interest income', 'control investments interest income', 'investment income from controlled affiliated investments interest income'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Controlled affiliated investments: Dividend income',
        'search_strings': ['from controlled affiliate company investments dividend income', 'from controlled affiliated investments dividend income', 'control investments dividend income', 'investment income from controlled affiliated investments dividend income'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Controlled affiliated investments: Payment-in-kind income',
        'search_strings': ['from controlled affiliate company investments payment in kind interest income', 'from controlled affiliated investments payment in kind interest income', 'control investments payment in kind interest income', 'investment income from controlled affiliated investments payment in kind interest income'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Controlled affiliated investments: Other income',
        'search_strings': ['from controlled affiliate company investments other income', 'from controlled affiliated investments fee income', 'from controlled affiliated investments other income', 'control investments other income', 'investment income from controlled affiliated investments other income'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Controlled affiliated investments: total interest, fee, and dividend income',
        'search_strings': ['interest fee and dividend income control investments'],
        'value1' : None,
        'value2' : None
    },
    {
        'row_title': 'Non-controlled affiliated investments: Interest income',
        'search_strings': ['from non controlled affiliate company investments interest income', 'from non controlled affiliated investments  interest income', 'affiliate investments interest income'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Non-controlled affiliated investments: Dividend income',
        'search_strings': ['from non controlled affiliate company investments dividend income', 'from non controlled affiliated investments dividend income', 'affiliate investments dividend income'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Non-controlled affiliated investments: Payment-in-kind income',
        'search_strings': ['from non controlled affiliate company investments payment in kind interest income', 'from non controlled affiliated investments payment in kind interest income', 'affiliate investments payment in kind interest income'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Non-controlled affiliated investments: Other income',
        'search_strings': ['from non controlled affiliate company investments other income', 'fee income', 'from non controlled affiliated investments other income', 'affiliate investments other income'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Non-controlled affiliated investments: total interest, fee, and dividend income',
        'search_strings': ['interest fee and dividend income affiliated investments'],
        'value1' : None,
        'value2' : None
    },
    {
        'row_title': 'Total interest income',
        'search_strings': ['investment income interest income'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Total PIK income',
        'search_strings': ['investment income payment in kind interest income'],
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Total dividend income',
        'search_strings': ['investment income dividend income'],
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Total fee income',
        'search_strings': ['investment income fee income'],
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Total investment income',
        'search_strings': ['total investment income'], 
        'value1': None,
        'value2': None
    },
]

type_of_income_list = [
    {
        'row_title': 'Total interest income',
        'search_strings': ['interest income'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Total dividend income',
        'search_strings': ['dividend income'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Total payment-in-kind income',
        'search_strings': ['payment in kind interest income'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Total other income',
        'search_strings': ['fee income'], 
        'value1': None,
        'value2': None
    },
]

type_of_investment_list = [
    {
        'row_title': 'Total non-controlled, non-affiliated investments',
        'search_strings': ['non control non affiliate investments'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Total controlled affiliated investments',
        'search_strings': ['control investments'], 
        'value1': None,
        'value2': None
    },
    {
        'row_title': 'Total non-controlled affiliated investments',
        'search_strings': ['affiliate investments'], 
        'value1': None,
        'value2': None
    },
]

# Create a DataFrame from the operations_list
desired_columns = ['row_title', 'value1', 'value2']
inv_df = pd.DataFrame([{key: d[key] for key in desired_columns} for d in investment_income_list])

print(inv_df)

                                            row_title value1 value2
0                                   Investment income   None   None
1   Non-controlled, non-affiliated investments: In...   None   None
2   Non-controlled, non-affiliated investments: Di...   None   None
3   Non-controlled, non-affiliated investments: Pa...   None   None
4   Non-controlled, non-affiliated investments: Ot...   None   None
5   Non-controlled, non-affiliated investments: to...   None   None
6   Controlled affiliated investments: Interest in...   None   None
7   Controlled affiliated investments: Dividend in...   None   None
8   Controlled affiliated investments: Payment-in-...   None   None
9     Controlled affiliated investments: Other income   None   None
10  Controlled affiliated investments: total inter...   None   None
11  Non-controlled affiliated investments: Interes...   None   None
12  Non-controlled affiliated investments: Dividen...   None   None
13  Non-controlled affiliated investments: Payme

In [231]:
cleaned_right = clean_text_2('investment income from non controlled non affiliated investments payment in kind interest income')
cleaned_actual = clean_text_2('Investment Income: Payment in-kind interest income')
print(fuzz.ratio(cleaned_right, cleaned_actual))

return_best_match_2('Investment Income: Payment in-kind interest income', investment_income_list)


# ['from non controlled non affiliate company investments payment in kind interest income',
#  'from non controlled non affiliated investments payment in kind interest income',
#  'non control non affiliate investments payment in kind interest income',
#  'investment income from non controlled non affiliated investments payment in kind interest income']

68


'Total PIK income'

My version (6.12.23) starts here

In [232]:
def clean_text_2 (row):
    no_punc = re.sub(r'[^\w\s]', '', row)
    lower = no_punc.lower()
    no_space = re.sub(r'\s+', '', lower)
    return no_space

clean_text_2 ('Hello 45!')

'hello45'

In [233]:
def average_score_2 (row, terms):
    # find average fuzzy score among all variations of one row title
    standard_row = clean_text_2 (row)
    total_score = 0
    for search_string in terms:
        score = fuzz.partial_ratio(standard_row, clean_text_2(search_string))
        total_score += score
    if len(terms) != 0:
        average_score = total_score / len(terms) 
    else:
        average_score = 0
    return average_score 

def return_best_match_2 (text, type_list):
    # return row title of best match
    text = clean_text_2(text)
    highest_average = 0
    winner = pd.NA
    for header in type_list:
        score = average_score_2(text, header['search_strings'])
        if score > highest_average:
            highest_average = score
            winner = header['row_title']
    for header in type_list:
        for search_string in header['search_strings']:
            if search_string == text or search_string in text:
                winner = header['row_title']
                
    return winner

In [234]:
df = pd.read_csv("blackstone_operations.csv")

# print(clean_text_2(df.iloc[:, 0][0]))

def extract_investment_rows (df, start_cell_str, end_cell_str):
    try:
        start_row = df[df.iloc[:, 0].apply(clean_text_2) == start_cell_str].index[0]
        end_row = df[df.iloc[:, 0].apply(clean_text_2) == end_cell_str].index[0]
    except IndexError:
        print("Start or end cell string not found in the DataFrame.")
        return
    
    portion = df.iloc[start_row : end_row + 1, :]
    return portion

In [235]:
df_investments = extract_investment_rows(df, 'investmentincome', 'totalinvestmentincome')
df_investments
# df_investments.iloc[1][0]

,Unnamed: 0,Three Months Ended 2023,Three Months Ended 2022
0,Investment income:,NaN,NaN
1,Interest income,254221000.0,170989000.0
2,Payment in-kind interest income,9841000.0,8686000.0
3,Dividend income,—,5908000.0
4,Fee income,876000.0,14000.0
5,Total investment income,264938000.0,185597000.0


In [236]:
def extract_subheadings (df):
    subheadings = []
    for i in range(len(df)):
        row = df.iloc[i]
        if pd.isnull(row[1]) and pd.isnull(row[2]):
            subheading = clean_text_2(row[0])
            subheadings.append(subheading)
    return subheadings
        
extract_subheadings(df_investments)

['investmentincome']

In [237]:
def get_investment_income (df):
    subheading = ''
    for i in range(len(df)):
        row = df.iloc[i]
        if pd.isnull(row[1]) and pd.isnull(row[2]):
            subheading = row[0]
            continue
        header = clean_text_2(subheading + row[0])
        best_match = return_best_match(header, investment_income_list)
        print(best_match)
        for item in investment_income_list:
                    if item['row_title'] == best_match:
                        item['value1'] = row[1]
                        item['value2'] = row[2]

In [238]:
header = clean_text_2('Interest, fee and dividend income:' + 'Control investments')
print(header)
return_best_match(header, investment_income_list)

interestfeeanddividendincomecontrolinvestments


'Controlled affiliated investments: total interest, fee, and dividend income'

In [239]:
get_investment_income (df_investments)
investment_income_list
# return_best_match_2(header, investment_income_list)
# investment_income_list

Total interest income
Non-controlled, non-affiliated investments: Payment-in-kind income
Total dividend income
Total fee income
Total investment income


[{'row_title': 'Investment income',
  'search_strings': [],
  'value1': None,
  'value2': None},
 {'row_title': 'Non-controlled, non-affiliated investments: Interest income',
  'search_strings': ['from non controlled non affiliate company investments interest income',
   'from non controlled non affiliated investments interest income',
   'non control non affiliate investments interest income',
   'investment income from non controlled  non affiliated investments interest income'],
  'value1': None,
  'value2': None},
 {'row_title': 'Non-controlled, non-affiliated investments: Dividend income',
  'search_strings': ['from non controlled non affiliate company investments dividend income',
   'from non controlled non affiliated investments dividend income',
   'non control non affiliate investments dividend income',
   'investment income from non controlled  non affiliated investments dividend income'],
  'value1': None,
  'value2': None},
 {'row_title': 'Non-controlled, non-affiliated in

In [128]:
def clean_text_op(row):
    # Strip out special characters/punction
    text = row.lower().strip()
    stripped_string = re.sub(r'[^a-zA-Z0-9\s]+', '', text)
    return stripped_string

def save_portion_of_table(csv_file, start_cell_str, end_cell_str):
    df = pd.read_csv(csv_file)

    # Find the indices of the starting and ending cells
    try:
        start_row = df[df.iloc[:, 0].apply(clean_text_op) == start_cell_str].index[0]
        end_row = df[df.iloc[:, 0].apply(clean_text_op) == end_cell_str].index[0]
    except IndexError:
        print("Start or end cell string not found in the DataFrame.")
        return
    
    # Extract the portion of the table based on the row indices
    portion = df.iloc[start_row:end_row+1, :]
    
    # Save the extracted portion to a new CSV file
    investment_income_append(portion)

# for loop to find rows with empty values, within it go down each row, append it to the empty value row, match it to truth table and put value into table
def investment_income_append(df):
    for index, row in df.iloc[1:].iterrows():
        if pd.isnull(row[1]) and pd.isnull(row[2]):
            #print(row[0])
            for index1, row1 in df.iloc[(index+1):].iterrows():
                if pd.isnull(row1[1]) and pd.isnull(row1[2]):
                    break
                header = return_best_match(row[0] + " " + row1[0], investment_income_list)
                first_value = row1[1]
                sec_value = row1[2]
                # Store the value in investment_income_list
                for item in investment_income_list:
                    if item['row_title'] == header:
                        item['value1'] = first_value
                        item['value2'] = sec_value


csv_file = 'fs_kkr_operations.csv'
start_cell_str = 'investment income'
end_cell_str = 'total investment income'

save_portion_of_table(csv_file, start_cell_str, end_cell_str)

# Create a DataFrame from the operations_list
desired_columns = ['row_title', 'value1', 'value2']
inv_df = pd.DataFrame([{key: d[key] for key in desired_columns} for d in investment_income_list])

print(inv_df)

NameError: name 'investment_income_list' is not defined

In [1240]:
def clean_text_op(row):
    # Strip out special characters/punction
    text = row.lower().strip()
    stripped_string = re.sub(r'[^a-zA-Z0-9\s]+', '', text)
    return stripped_string

def save_portion_of_table(csv_file, start_cell_str, end_cell_str):
    df = pd.read_csv(csv_file)

    # Find the indices of the starting and ending cells
    try:
        start_row = df[df.iloc[:, 0].apply(clean_text_op) == start_cell_str].index[0]
        end_row = df[df.iloc[:, 0].apply(clean_text_op) == end_cell_str].index[0]
    except IndexError:
        print("Start or end cell string not found in the DataFrame.")
        return
    
    # Extract the portion of the table based on the row indices
    portion = df.iloc[start_row:end_row+1, :]
    
    # Save the extracted portion to a new CSV file
    investment_income_append(portion)

# for loop to find rows with empty values, within it go down each row, append it to the empty value row, match it to truth table and put value into table
def investment_income_append(df):
    if pd.isna(df.iloc[1, 1]) and pd.isna(df.iloc[1, 2]):
        if df.iloc[1, 0] == 'Interest, fee and dividend income:':
            for index, row in df.iloc[2:].iterrows():
                for header in investment_income_list:
                    for search_string in header['search_strings']:
                        if search_string == clean_text(row[0]):
                            winner = header['row_title']
                            first_value = row[1]
                            sec_value = row[2]
                            # Store the value in investment_income_list
                            for item in investment_income_list:
                                if item['row_title'] == winner:
                                    item['value1'] = first_value
                                    item['value2'] = sec_value
        else:
            for index, row in df.iloc[1:].iterrows():
                for index1, row1 in df.iloc[(index+1):].iterrows():
                    if pd.isnull(row1[1]) and pd.isnull(row1[2]):
                        break
                    header = return_best_match(row[0] + " " + row1[0], investment_income_list)
                    first_value = row1[1]
                    sec_value = row1[2]
                    # Store the value in investment_income_list
                    for item in investment_income_list:
                        if item['row_title'] == header:
                            item['value1'] = first_value
                            item['value2'] = sec_value
    else:
        for index, row in df.iloc[1:].iterrows():
            for header in investment_income_list:
                for search_string in header['search_strings']:
                    if search_string == clean_text(row[0]):
                        winner = header['row_title']
                        first_value = row[1]
                        sec_value = row[2]
                        # Store the value in investment_income_list
                        for item in investment_income_list:
                            if item['row_title'] == winner:
                                item['value1'] = first_value
                                item['value2'] = sec_value



csv_file = 'fs_kkr_operations.csv'
start_cell_str = 'investment income'
end_cell_str = 'total investment income'

save_portion_of_table(csv_file, start_cell_str, end_cell_str)

# Create a DataFrame from the operations_list
desired_columns = ['row_title', 'value1', 'value2']
inv_df = pd.DataFrame([{key: d[key] for key in desired_columns} for d in investment_income_list])

print(inv_df)

                                            row_title       value1  \
0                                   Investment income         None   
1   Non-controlled, non-affiliated investments: In...         None   
2   Non-controlled, non-affiliated investments: Di...         None   
3   Non-controlled, non-affiliated investments: Pa...   10000000.0   
4   Non-controlled, non-affiliated investments: Ot...         None   
5   Controlled affiliated investments: Interest in...   19000000.0   
6   Controlled affiliated investments: Dividend in...         None   
7   Controlled affiliated investments: Payment-in-...         None   
8     Controlled affiliated investments: Other income         None   
9   Non-controlled affiliated investments: Interes...    3000000.0   
10  Non-controlled affiliated investments: Dividen...   63000000.0   
11  Non-controlled affiliated investments: Payment...   10000000.0   
12  Non-controlled affiliated investments: Other i...   63000000.0   
13                  

In [1241]:
# Merge the DataFrames based on a common column
inv_df['value1'] = inv_df['value1'].astype(object)
inv_df['value2'] = inv_df['value2'].astype(object)

op_df['value1'] = op_df['value1'].astype(object)
op_df['value2'] = op_df['value2'].astype(object)

# Concatenate the DataFrames
merged_df = pd.concat([inv_df, op_df], ignore_index=True)
#merged_df = pd.merge(inv_df, op_df, on=['row_title', 'value1', 'value2'], how='outer')
print(merged_df)

                                            row_title       value1  \
0                                   Investment income         None   
1   Non-controlled, non-affiliated investments: In...         None   
2   Non-controlled, non-affiliated investments: Di...         None   
3   Non-controlled, non-affiliated investments: Pa...   10000000.0   
4   Non-controlled, non-affiliated investments: Ot...         None   
5   Controlled affiliated investments: Interest in...   19000000.0   
6   Controlled affiliated investments: Dividend in...         None   
7   Controlled affiliated investments: Payment-in-...         None   
8     Controlled affiliated investments: Other income         None   
9   Non-controlled affiliated investments: Interes...    3000000.0   
10  Non-controlled affiliated investments: Dividen...   63000000.0   
11  Non-controlled affiliated investments: Payment...   10000000.0   
12  Non-controlled affiliated investments: Other i...   63000000.0   
13                  